In [2]:
%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../../")

import anamic

import numpy as np
import matplotlib.pyplot as plt
import tqdm
from joblib import Parallel, delayed

data_dir = Path('/home/hadim/Drive/Data/Fake Microtubules/anamic/taper_length')
data_dir.mkdir(parents=True, exist_ok=True)

/home/hadim/local/conda/envs/ws/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
../../../anamic/transformations.py:1916: UserWarning: No module named 'anamic._transformations'
  warnings.warn(str(err))


In [5]:
taper_lengths = [0, 500, 1000, 1500, 2000, 2500, 3000]
n_mt_per_condition = 50

# We choose the pf number from the described distribution
# in Chaaban et al., 2018, Fig. 4B.
all_n_pf = [11, 12, 13, 14, 15]
n_pf_proba = [0.05, 0.05, 0.3, 0.1, 0.5]

mt_length_nm = 6000  # nm

# Set parameters for the image generation.
parameters = {}
parameters['labeling_ratio'] = 0.1  # from 0 to 1

parameters['pixel_size'] = 110  # nm/pixel
parameters['x_offset'] = 4500  # nm
parameters['y_offset'] = 4500  # nm

parameters['psf_size'] = 130  # nm

parameters['signal_mean'] = 700
parameters['signal_std'] = 100
parameters['bg_mean'] = 500
parameters['bg_std'] = 24
parameters['noise_factor'] = 1.2

parameters['snr_line_width'] = 3  # pixel

worker_args = []
for taper_length in taper_lengths:
    current_dir = data_dir / f'taper_length_nm-{taper_length:04d}'
    current_dir.mkdir(parents=True, exist_ok=True)
    for i in range(n_mt_per_condition):
        basename = f'microtubule_{i:04d}'
        worker_args.append({'taper_length': taper_length, 'i_mt': i,
                            'current_dir': current_dir, 'basename': basename})
        
def worker(taper_length, i_mt, current_dir, basename):
    
    n_pf = np.random.choice(all_n_pf, 1, p=n_pf_proba)[0]
    
    # Generate the dimers
    dimers = anamic.simulator.dimers_builder(n_pf, mt_length_nm, taper_length)
    
    # Create the simulator object
    ms = anamic.simulator.MicrotubuleSimulator(dimers)
    
    # Update the parameters
    ms.parameters.update(parameters)
        
    # Build a single microtubule image
    ms.build_all(apply_random_z_rotation=True, show_progress=False)
    
    # Save image, parameters and dimer positions.
    ms.save_positions(current_dir / (basename + ".csv"))
    ms.save_parameters(current_dir / (basename + ".json"))
    ms.save_image(current_dir / (basename + ".tif"))

executor_factory = anamic.utils.parallel_executor(use_bar='tqdm', n_jobs=-1)(total=len(worker_args))
executor = executor_factory(delayed(worker)(**worker_arg) for worker_arg in worker_args)